In [1]:
import anthropic
from datasets import load_dataset

client = anthropic.Anthropic()
dataset = load_dataset("AI4Math/MathVista")

c:\Users\axeld\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def get_prompt(query:str, answer:str, generated_answer:str) -> str:
    prompt = """Given following MathVista query, answer and generated answer, output "yes" or "no" regarding whether the generated answer is correct.
    Query: <query> {query} </query>
    Answer: <answer> {answer} </answer>
    Generated Answer: <generated_answer> {generated_answer} </generated_answer>

    Output only "yes" or "no" regarding correctness of the generated answer.
    Output:""".format(query=query, answer=answer, generated_answer=generated_answer)
    return prompt

## Saving only right answers

In [12]:
import pandas as pd

df = pd.read_csv("training_data/training_v2.csv")
print(df.iloc[0])

questions            Hint: Please answer the question requiring a f...
thoughts             The problem describes a scenario where a canis...
generated_answers    Solution:\nThe initial kinetic energy of the c...
true_answers                                                       1.2
indexes                                                              0
Name: 0, dtype: object


In [19]:
validity_list = []
for i in range(len(df)):
    print(f"step {i}")
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": get_prompt(query=df.iloc[i]["questions"], answer=df.iloc[i]["true_answers"], generated_answer=df.iloc[i]["generated_answers"])
                    }
                ]
            }
        ]
    )
    validity_list.append(message.content[0].text)

step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
step 101
step 102
step 103
step 104
step 105
step 106
step 107
step 108
step 109
step 110
step 111
step 112
step 113
step 114
step 115
step 116
step 117
step 118
step 119
step 120
step 121
step 122
ste

In [20]:
df["validity"] = validity_list
print(len(df[df["validity"] == "yes"]) / len(df))

0.7275362318840579


In [21]:
df[df["validity"] == "yes"].to_csv("training_v2_correct.csv", index=False)